In [ ]:
from transformers import pipeline
from PIL import Image
from diffusers import StableDiffusionPipeline
import torch

# Use a smaller text model for CPU
text_generator = pipeline("text-generation", model="distilgpt2", max_length=200)

# Set device to CPU explicitly
device = "cpu"

# Load a CPU-friendly image generation model
image_generator = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)

def generate_image(prompt):
    try:
        print("Generating image, please wait...")
        image = image_generator(prompt).images[0]
        image_path = "generated_image.png"
        image.save(image_path)
        return f"Chatbot: Image saved as {image_path}"
    except Exception as e:
        return f"Error generating image: {e}"

# Function to generate text
def generate_text(prompt):
    try:
        response = text_generator(prompt, max_length=200, num_return_sequences=1)
        return response[0]['generated_text']
    except Exception as e:
        return f"Error generating text: {e}"

# Multi-modal chatbot
def multimodal_chatbot():
    print("Welcome to the Multi-Modal Chatbot! Type 'exit' to quit.")

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Image generation
        if "generate image" in user_input.lower():
            prompt = user_input.replace("generate image", "").strip()
            if prompt:
                print("Chatbot: Generating an image, please wait...")
                image_path = generate_image(prompt)
                print(f"Chatbot: Here's your image saved as {image_path}" if "Error" not in image_path else image_path)
            else:
                print("Chatbot: Please provide a description for the image.")

        # General text processing
        else:
            print("Chatbot: Processing your query...")
            response = generate_text(user_input)
            print(f"Chatbot: {response}")

# Run the chatbot
if __name__ == "__main__":
    multimodal_chatbot()
